In [1]:
import requests
import pandas as pd
import numpy as np
import json
import os
# import glob
from pathlib import Path
import re
import time
from datetime import datetime
from collections import Counter
from IPython.core.debugger import set_trace

from tqdm.auto import tqdm
tqdm.pandas()

import os
import sys
parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)

C:\Users\infomax\Documents\GitHub\getch\venv\lib\site-packages\tqdm\std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
import django
from django.db import connection
from django.core.files.base import ContentFile
django.setup()

from getchapp.models import User, Avatar, Pix, Brand, Item, Post, Tag

# DB 초기화

In [3]:
def logo_file(bname, where='uploads/brand'):
    try:
        return list(Path(where).glob(bname + '.*'))[-1]
    except:
        return None
    
logo_file('nike')

WindowsPath('uploads/brand/nike.jpg')

In [4]:
getch_master = User.objects.get(email='get.ch@getch.com'); getch_master

<User: get.ch@getch.com>

# Brand

In [6]:
ibrands = list(pd.read_excel('keywords and logos.xlsx', sheet_name='20190430').itertuples())[3:]

for row in tqdm(sorted(ibrands, key=lambda x:x.shortname)[::-1]):
    logo = logo_file(row.shortname)
    
    if logo is not None:
        opt = {
            'name': row.shortname,
            'keywords': row.keywords,
            'master': getch_master, 
            'fullname_en': row.fullname,
            'fullname_kr': row.koname,
            'origin': row.origin,
            'category': row.category.lower(), 
            'text': ''
        }
        
        brand = Brand.objects.create(**opt)
        avatar = Avatar.objects.create(owner=getch_master, default_on=brand)
        avatar.src.save(logo.name, ContentFile(logo.read_bytes()), save=True)
        
        brand.avatar = avatar
        brand.save()
        
    else:
        print(row.shortname)

onlineceramics



# Item

In [7]:
items = [
    {'name': 'skirt', 'keywords': 'skirt, skirts, 스커트, 치마', 'image':'uploads/item/skirt.png'},
    {'name': 'shoe', 'keywords': 'shoe, shoes, 슈즈, 신발', 'image':'uploads/item/shoe.png'},
    {'name': 'bottom', 'keywords': 'bottom, bottoms, 하의', 'image':'uploads/item/bottom.png'},
    {'name': 'outer', 'keywords': 'outer, outers, 아우터', 'image':'uploads/item/outer.png'},
    {'name': 'top', 'keywords': 'top, tops, 상의', 'image':'uploads/item/top.png'},
    {'name': 'dress', 'keywords': 'dress, 드레스, 원피스', 'image':'uploads/item/dress.png'},
    {'name': 'bag', 'keywords': 'bag, bags, 백, 가방', 'image':'uploads/item/bag.png'},
    {'name': 'accessory', 'keywords': 'accessory, accessories, 악세서리, 장신구', 'image':'uploads/item/accessory.png'},
]

for _item in tqdm(items):
    opt = {
        'name': _item['name'],
        'keywords': _item['keywords'],
        'master': getch_master, 
        'text': ''
    }
    item = Item.objects.create(**opt)
    avatar = Avatar.objects.create(owner=getch_master, default_on=item)
    avatar.src.save(_item['name']+'.png', ContentFile(Path(_item['image']).read_bytes()), save=True)
    
    item.avatar = avatar
    item.save()

# Post

In [3]:
sejin76 = User.objects.get(email='sejin76@nate.com'); sejin76

<User: sejin76@nate.com>

In [4]:
posts = [
    {'image':'uploads/post/samples/15.jpg', 'text':'벤 하디 보헤미안 랩소디에 나온 금발머리 형'},
    {'image':'uploads/post/samples/16.jpg', 'text':'이런 건 좀 기사에  올리지 마라 저거 보고 따라 하다 진짜 불상사 생기면 누가 책임 질래 적당히 좀 해라 개는 개일 뿐'},
    {'image':'uploads/post/samples/17.jpg', 'text':'굉장히 위험해보임 ㅠ'},
    {'image':'uploads/post/samples/18.jpg', 'text':'아무리 귀여워도 동물은 자기들끼리 위계질서가 있어. 자기보다 약하다고 생각하는 순간 계속 괴롭히고 공격한다. 그래서 애완동물은 교육을 잘 시켜야 한다. 동물은 그냥 동물일뿐이다.'},
    {'image':'uploads/post/samples/19.webp', 'text':'개를 개로 키워야지 뇌 구조가 사람으로 발달되는 건 개 한테도 불행합니다 강형욱씨 프로그램에서 내내 강조하는 내용입니다 한심하네.... 기자야 에구 뭘 좀 알고 삽시다'},
    {'image':'uploads/post/samples/20.jpg', 'text':'토착왜구 추악한 늙은이 기업 꼴마?'},
    {'image':'uploads/post/samples/21.jpg', 'text':'술을 안먹어야지 ㅋㅋ'},
    {'image':'uploads/post/samples/22.jpg', 'text':'중고딩때 정말 예뻤네 나라도 한번보러갈듯'},
    {'image':'uploads/post/samples/23.jpg', 'text':'졸업앨범에 같이 찍힌 일반인들 좀 가려주지ㅜㅠㅜㅜㅠㅠㅠㅠ....... 일반인인데 영원히 고통 받겠네......'},
    {'image':'uploads/post/samples/24.jpg', 'text':'화장 안한게 더 이뻐~'},
    {'image':'uploads/post/samples/25.jpg', 'text':'연예인 사진만 편집해서 올리지 옆 친구는 왜 올리는 거죠'},
    {'image':'uploads/post/samples/26.jpg', 'text':'일산정보고 출신들 댓글좀 달아봐 그정도는 아니였던거 같은데'},
    {'image':'uploads/post/samples/27.jpg', 'text':'난 남잔데 전혀 아닌데 이쁘긴 하지만 평범한데...나만 그런가...한번도 매력 있다고 생각해 본적 없는데..'},
    {'image':'uploads/post/samples/28.jpg', 'text':'국제 롤라장에서 본듯해~ㅋㅋ'},
    {'image':'uploads/post/samples/29.jpg', 'text':'구라치면 디져유'},
    {'image':'uploads/post/samples/30.jpg', 'text':'멋있다'},
    {'image':'uploads/post/samples/31.jpg', 'text':'어후~ 어떻게 저 패션이 28년전 패션이냐.... 요즘도 아주 패셔너블한 힙한 패션인데 진짜 신기함.'},
    {'image':'uploads/post/samples/32.jpg', 'text':'예쁘다♡♡'},
    {'image':'uploads/post/samples/33.jpg', 'text':'사진도 기사도 예쁜데 50대 초반에게 늙었다는 표현은  좀 아닌 듯 하네요.. ^^'},
    {'image':'uploads/post/samples/34.jpg', 'text':'캬~~~아'},
    {'image':'uploads/post/samples/35.jpg', 'text':'패션화보 안찍나요?   독특하면서 기발한 작품 입혀주세요~'},
]

for _post in tqdm(posts):
    pix = Pix.objects.create(owner=sejin76)
    pix.src.save(_post['image'].split('/')[-1], ContentFile(Path(_post['image']).read_bytes()), save=True)
    
    opt = {
        'name': sejin76.name + '__' + str(datetime.now()),
        'master': sejin76,
        'text': _post['text'],
        'pix': pix,
    }
    
    Post.objects.create(**opt)

In [ ]:
import os
import shutil
 
path = 'C:/Users/kyeongrok.kim/Desktop/hello/aaa/'
resultPath = 'C:/Users/kyeongrok.kim/Desktop/hello/bbb/'
 
list = os.listdir(path)
 
def getfiles(dirpath):
    a = [s for s in os.listdir(dirpath) if os.path.isfile(os.path.join(dirpath, s))]
    a.sort(key=lambda s: os.path.getmtime(os.path.join(dirpath, s)))
    return a
 
count = 1
for filename in list:
    fromFilePathName = path + filename
    resultFilePathName = resultPath + str(count) + '.json'
    shutil.move(fromFilePathName, resultFilePathName)
 
    print(fromFilePathName)
    print(resultFilePathName)
    count += 1

In [3]:
from pathlib import Path

In [9]:
for file in Path('uploads/brand').glob('*'):
    print(file.read_bytes())
    break

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x00\xff\xfe\x00;CREATOR: gd-jpeg v1.0 (using IJG JPEG v80), quality = 92\n\xff\xdb\x00C\x00\x03\x02\x02\x02\x02\x02\x03\x02\x02\x02\x03\x03\x03\x03\x04\x06\x04\x04\x04\x04\x04\x08\x06\x06\x05\x06\t\x08\n\n\t\x08\t\t\n\x0c\x0f\x0c\n\x0b\x0e\x0b\t\t\r\x11\r\x0e\x0f\x10\x10\x11\x10\n\x0c\x12\x13\x12\x10\x13\x0f\x10\x10\x10\xff\xdb\x00C\x01\x03\x03\x03\x04\x03\x04\x08\x04\x04\x08\x10\x0b\t\x0b\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\xff\xc0\x00\x11\x08\x01\xe0\x01\xe0\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\

In [2]:
'{a}-{b}-{c}'.format(a=1,b=2,c=3,d=4)

'1-2-3'